<a href="https://colab.research.google.com/github/lhccd/mini_projects/blob/main/CSS_SemanticDrift_lorenz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json

with open('/content/drive/MyDrive/Colab Notebooks/ukraine_jan_2013_to_jan_2016.json', 'r') as f:
  data = json.load(f)


In [3]:
txt = ""

for tweet in data: 
  txt = txt + tweet["tweet_text"]

print(txt[:1000])

"I CAN ONLY SAY UKRAINE THIS IS YOUR LAND AND YOUR PEOPLE SO DEAL WITH RUSSIA LIKE YOU WOULD DEAL WITH A BUSINESS... https://t.co/kP2MUpnv2OBetrayal or Victory: what did 2015 bring for #Ukraine? 
https://t.co/jg50T2qVGj https://t.co/zLs07uOh9J#Lviv Jan 01 01:30 Temperature -13C cloudless, Haze Humidity 90% Ukraine .. https://t.co/6HvshVbRMuUkraine: Russia hacking power plants, highlighting U.S. weakness https://t.co/qUha2LNN3L@USEmbRu The best solution to the problems of Ukraine#60dc2 #60dc How To Celebrate Ukraine New Year's Traditions For 20 Days https://t.co/ybMSBdtXiOUkrainian economy in ruins after neocon coup. Where's State Mama Nudelman with her free cookies now? asked W Madsen
https://t.co/Sx5OTbmzwLRussia and Ukraine Finally Break Up - Bloomberg View https://t.co/Jc6pSbGUwmPower cuts in Crimea after new blast on Ukraine power line #power https://t.co/m4RYdGPzYM@alkapranos @B0bHardy Happy New Year from Ukraine! :) https://t.co/ixO7CXXksCHappy New Year from Ukraine 🎄🎉Spotlight: 

In [4]:
# perform data cleaning
## TODO: remove links/ user mentions

# lower case txt
txt = txt.lower()

# remove emojis
import re
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

txt = deEmojify(txt)

In [5]:
# save text file

#open text file
text_file = open('/content/train.txt', 'w')
 
#write string to file
text_file.write(txt)
 
#close file
text_file.close()

In [6]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
with open('/content/train.txt', 'r') as f:
  tweets_text = f.read()

print(tweets_text[:1000])
testset = tweets_text[:10000]
word_tokenize(testset)

In [ ]:
!pip install transformers
!pip install tokenizers

from pathlib import Path
from tokenizers import ByteLevelBPETokenizer

In [15]:
paths = ['/content/train.txt']

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

!mkdir Ukraine_ML
tokenizer.save_model("Ukraine_ML")


['Ukraine_ML/vocab.json', 'Ukraine_ML/merges.txt']

In [16]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

tokenizer = ByteLevelBPETokenizer(
    "./Ukraine_ML/vocab.json",
    "./Ukraine_ML/merges.txt",
)

In [17]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [18]:
tokenizer.encode("This is putin").tokens

['<s>', 'T', 'his', 'Ġis', 'Ġputin', '</s>']

Train Language Model

In [19]:
import torch
torch.cuda.is_available()

False

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)